In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from pathlib import Path
from datetime import datetime
import os
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import requests
import json

s=requests.Session()
s.headers = {
    'authority': 'homes.hdb.gov.sg',
    'sec-ch-ua': '" Not;A Brand";v="99", "Google Chrome";v="91", "Chromium";v="91"',
    'accept': 'application/json, text/plain, */*',
    'sessionid': 'a',
    'sec-ch-ua-mobile': '?0',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.77 Safari/537.36',
    'content-type': 'application/json',
    'origin': 'https://homes.hdb.gov.sg',
    'sec-fetch-site': 'same-origin',
    'sec-fetch-mode': 'cors',
    'sec-fetch-dest': 'empty',
    'referer': 'https://homes.hdb.gov.sg/home/finding-a-flat',
    'accept-language': 'en-US,en;q=0.9,zh-CN;q=0.8,zh;q=0.7',
}


In [3]:

data = '{"location":"","coordinates":[["",""]],"range":"2","priceRangeLower":"0","priceRangeUpper":"0","flatType":"","waitingTime":"","modeOfSale":"BTO,SBF","floorRange":"","remainingLeaseRangeLower":1,"remainingLeaseRangeUpper":99,"ethnicGroup":"Chinese","rank":"Location, Price Range, Flat Type, Remaining Lease"}'

response = s.post('https://homes.hdb.gov.sg/home-api/public/v1/map/getCoordinatesByFilters', data=data)

response


<Response [200]>

In [14]:
jsonstr

[{'coordinates': '[1.3972173, 103.7458813]',
  'properties': {'address': 'Heart of Yew Tee',
   'listingType': 'BTO',
   'description': {'photoUrl': '2021-11/bto/cck_n6c5a/cck_n6c5a_thumbnail.jpg',
    'maxRemainingLease': '45',
    'stage': 'Launch',
    'distance': '-1',
    'town': 'Choa Chu Kang',
    'launchStartDate': '2021-11-17 13:30:00.0',
    'ballotQtr': '2021-11',
    'flatType': '2-Room Flexi',
    'maxPrice': '98000',
    'projectId': '2021-11_BTO_Q0NLX042QzVBXzE2MzQ3OTYwNjM1NzQ'},
   'hdbCategory': '1',
   'region': 'WEST REGION'},
  'details': '<?xml version="1.0" encoding="UTF-8"?>\n<template>\n    <project-name>Heart of Yew Tee</project-name>\n    <standard-project-info>\n        <key>\n            <town>Choa Chu Kang</town>\n            <tooltip/>\n            <uri/>\n        </key>\n        <key>\n            <point>1050|1101</point>\n        </key>\n        <key>\n            <latitude>1.3972173</latitude>\n            <longitude>103.7458813</longitude>\n        </

In [4]:
jsonstr=json.loads(response.text)
detailedUrl='https://resources.homes.hdb.gov.sg/nf/{}.xml'
# https://resources.homes.hdb.gov.sg/nf/2021-05/bto/tg_d2c4/2021-05_BTO_VEdfRDJDNF8xNjE5NTAyODQxMDYz.xml
#all project details xml link follow this format - works well for BTO
for x in jsonstr:
    storageDict={}
#     to try and reduce API calls on price range
    x['properties']['description']=x['properties']['description'][0]
    if x['properties']['listingType']=='BTO':
        projPath=x['properties']['description'].get('photoUrl','').rpartition('/')[0]
        newUrl=detailedUrl.format(projPath+'/'+x['properties']['description']['projectId'])
    elif x['properties']['listingType']=='SBF':
        ballotQtr=x['properties']['description']['ballotQtr']
        projectID=x['properties']['description']['projectId']
        projDict='{"projectId":"'+projectID+'"}'
        storageDict={}
        x['availability']=json.loads(s.post('https://homes.hdb.gov.sg/home-api/public/v1/launch/getProjectAvailabilityAndEthnic',data=projDict).text)
        for block,value in x['availability']['projectBlockFlatTypeInfoMap'].items():
            for flatType in value.keys():
                if storageDict.get(flatType) is None:
                    priceDict='{"projectId":"'+projectID+'","flatType":"'+flatType+'"}'
                    storageDict[flatType]=s.post('https://homes.hdb.gov.sg/home-api/public/v1/launch/getPriceRangeAcrossTownByProjectIdAndFlatType',data=priceDict).text
                x['availability']['projectBlockFlatTypeInfoMap'][block][flatType]['areaPrice']=storageDict[flatType]
                    
        newUrl=detailedUrl.format(ballotQtr+'/sbf/unit_xml/UNIT_'+projectID)
    details=s.get(newUrl)
    x['details']=details.text

In [5]:
df=pd.json_normalize(jsonstr,max_level=1)
df.head(5)

,coordinates,details,properties.address,properties.listingType,properties.description,properties.hdbCategory,properties.region,availability.availabilitySet,availability.projectBlockFlatTypeInfoMap
0,"[1.3972173, 103.7458813]","<?xml version=""1.0"" encoding=""UTF-8""?>\n<templ...",Heart of Yew Tee,BTO,{'photoUrl': '2021-11/bto/cck_n6c5a/cck_n6c5a_...,1,WEST REGION,NaN,NaN
1,"[1.3049589, 103.855994]","<?xml version=""1.0"" encoding=""UTF-8""?>\n<templ...",River Peaks I & II,BTO,{'photoUrl': '2021-11/bto/ct_n5c7-c8/ct_n5c7-c...,1,CENTRAL REGION,NaN,NaN
2,"[1.3173528, 103.8504285]","<?xml version=""1.0"" encoding=""UTF-8""?>\n<templ...",Kent Heights,BTO,{'photoUrl': '2021-11/bto/kwn_n1c59/kwn_n1c59_...,1,CENTRAL REGION,NaN,NaN
3,"[1.3485867, 103.887738]","<?xml version=""1.0"" encoding=""UTF-8""?>\n<templ...",Tanjong Tree Residences @ Hougang,BTO,{'photoUrl': '2021-11/bto/hg_n1c13/hg_n1c13_th...,1,NORTH-EAST REGION,NaN,NaN
4,"[1.3486457, 103.888666]","<?xml version=""1.0"" encoding=""UTF-8""?>\n<templ...",Hougang Olive,BTO,{'photoUrl': '2021-11/bto/hg_n1c14/hg_n1c14_th...,1,NORTH-EAST REGION,NaN,NaN


In [6]:
df['details'][5]

'<?xml version="1.0" encoding="UTF-8"?>\n<template>\n    <project-name>Nanyang Opal</project-name>\n    <standard-project-info>\n        <key>\n            <town>Jurong West</town>\n            <tooltip/>\n            <uri/>\n        </key>\n        <key>\n            <point>907|1100</point>\n        </key>\n        <key>\n            <latitude>1.3406603</latitude>\n            <longitude>103.6909658</longitude>\n        </key>\n        <key>\n            <completion-date>4Q2025</completion-date>\n            <tooltip>\n                \n        This is an estimated date that the flat will be completed and ready for key collection, and the actual completion date will depend on the progress of construction works on site. As the Estimated Completion Date (ECD) is indicative only, HDB shall not be responsible for any damage or loss arising from a change in the ECD.\n        &lt;br/>&lt;br/>\n        After booking the flat, buyers may check for updates via My HDBPage on HDB InfoWEB, as the

In [7]:
# create past data save path
monthYear = datetime.now().strftime('%h-%Y').lower()
year = datetime.now().strftime('%Y')
savePath = os.path.join('past_data', year, monthYear)
Path(savePath).mkdir(parents=True, exist_ok=True)

In [11]:
# save data in various formats
fileName = os.path.join(savePath, 'hdb-sale-'+monthYear)
df.to_hdf(fileName+'.h5','HDBSALES')
df.to_pickle(fileName+'.pkl')
with open(fileName+".json", "w") as write_file:
    json.dump(jsonstr, write_file)

In [16]:
jsonstr[1]

{'coordinates': '[1.3049589, 103.855994]',
 'properties': {'address': 'River Peaks I & II',
  'listingType': 'BTO',
  'description': {'photoUrl': '2021-11/bto/ct_n5c7-c8/ct_n5c7-c8_thumbnail.jpg',
   'maxRemainingLease': '99',
   'stage': 'Launch',
   'distance': '-1',
   'town': 'Central',
   'launchStartDate': '2021-11-17 13:30:00.0',
   'ballotQtr': '2021-11',
   'flatType': '3-Room, 4-Room',
   'maxPrice': '688000',
   'projectId': '2021-11_BTO_Q1RfTjVDNy1DOF8xNjM1MzE1NTQzNDEx'},
  'hdbCategory': '1',
  'region': 'CENTRAL REGION'},
 'details': '<?xml version="1.0" encoding="UTF-8"?>\n<template>\n    <project-name>River Peaks I &amp; II</project-name>\n    <standard-project-info>\n        <key>\n            <town>Central</town>\n            <tooltip/>\n            <uri/>\n        </key>\n        <key>\n            <point>1517|984</point>\n        </key>\n        <key>\n            <latitude>1.3049589</latitude>\n            <longitude>103.8559940</longitude>\n        </key>\n       